In [23]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import ipywidgets as widgets
from IPython.display import display

In [24]:
data=pd.read_csv('data.csv')
print(data.shape)
data.sample(5)

(7445, 9)


,Date,Price,Open,High,Low,Vol.,Change %,Ticker Symbol,Bank Name
2769,04/03/2019,"27,322.8","26,633.9","27,628.9","26,366.1",2.38M,2.59%,BID,BIDV
3392,09/19/2022,"25,500.0","25,950.0","26,150.0","25,250.0",2.35M,-2.67%,CTG,Vietinbank
4119,10/22/2019,"16,104.8","15,843.8","16,104.8","15,843.8",2.48M,1.65%,CTG,Vietinbank
698,07/29/2021,"75,793","75,406","75,793","75,096",1.52M,0.93%,VCB,Vietcombank
6735,04/02/2021,"12,106.1","12,745.8","12,745.8","12,007.6",1.88M,-1.60%,AGR,Agribank


In [25]:
# Chuyển đổi cột 'Date' sang kiểu dữ liệu datetime
data['Date'] = pd.to_datetime(data['Date'])
# Chuyển đổi các cột số sang kiểu dữ liệu số
numeric_columns = ['Price', 'Open', 'High', 'Low', 'Change %']
for col in numeric_columns:
    data[col] = data[col].astype(str).str.replace(',', '').str.replace('%', '')
    data[col] = pd.to_numeric(data[col], errors='coerce')

def convert_volume(vol):
    if vol.endswith('M'):
        return float(vol[:-1]) * 1_000_000
    elif vol.endswith('K'):
        return float(vol[:-1]) * 1_000
    else:
        return float(vol)

data['Vol.'] = data['Vol.'].apply(convert_volume)

# Kiểm tra loại dữ liệu của các cột
print(data.dtypes)

data.sample(5)

Date             datetime64[ns]
Price                   float64
Open                    float64
High                    float64
Low                     float64
Vol.                    float64
Change %                float64
Ticker Symbol            object
Bank Name                object
dtype: object


,Date,Price,Open,High,Low,Vol.,Change %,Ticker Symbol,Bank Name
279,2023-04-04,92000.0,92800.0,92900.0,91600.0,269900.0,-0.86,VCB,Vietcombank
1655,2023-09-15,46800.0,47000.0,47400.0,46800.0,1050000.0,0.11,BID,BIDV
1606,2023-11-23,43000.0,44000.0,44000.0,43000.0,755300.0,-1.38,BID,BIDV
7207,2019-05-20,3720.4,3572.8,3789.3,3572.8,173160.0,4.13,AGR,Agribank
5496,2020-04-06,16950.0,16200.0,16950.0,16100.0,2970000.0,6.94,TCB,Techcombank


### Câu hỏi 1: Giá cổ phiếu thay đổi theo thời gian như thế nào?

In [26]:
# Biểu đồ giá cổ phiếu theo thời gian
fig = px.line(data, x='Date', y='Price', color='Ticker Symbol', title='Biểu đồ giá cổ phiếu theo thời gian')
fig.show()

In [27]:
# Kiểm định hồi quy tuyến tính để xem mối quan hệ giữa thời gian và giá cổ phiếu
print("\nHồi quy tuyến tính cho từng cổ phiếu theo thời gian:")
for stock in data['Ticker Symbol'].unique():
    stock_data = data[data['Ticker Symbol'] == stock]
    stock_data['Date_ordinal'] = stock_data['Date'].map(pd.Timestamp.toordinal)
    model = ols('Price ~ Date_ordinal', data=stock_data).fit()
    print(f'\nHồi quy tuyến tính cho {stock}:')
    print(model.summary())


Hồi quy tuyến tính cho từng cổ phiếu theo thời gian:

Hồi quy tuyến tính cho VCB:
                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.841
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     7877.
Date:                Tue, 21 May 2024   Prob (F-statistic):               0.00
Time:                        13:58:59   Log-Likelihood:                -15150.
No. Observations:                1489   AIC:                         3.030e+04
Df Residuals:                    1487   BIC:                         3.031e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

C:\Users\PC\AppData\Local\Temp\ipykernel_11208\3052835823.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\PC\AppData\Local\Temp\ipykernel_11208\3052835823.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\PC\AppData\Local\Temp\ipykernel_11208\3052835823.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

### Câu hỏi 2: So sánh giá cổ phiếu giữa các ngân hàng

In [28]:
# Biểu đồ phân phối giá đóng cửa của từng cổ phiếu
fig = px.histogram(data, x='Price', color='Ticker Symbol', nbins=50, marginal='rug', title='Phân phối giá đóng cửa của từng cổ phiếu')
fig.show()

In [29]:
# Kiểm định t-test cho hai nhóm cổ phiếu
# Ví dụ: So sánh giá cổ phiếu của VCB và BID
vcb_price = data[data['Ticker Symbol'] == 'VCB']['Price']
bid_price = data[data['Ticker Symbol'] == 'BID']['Price']
t_stat, p_val, df = ttest_ind(vcb_price, bid_price)
print(f'\nKiểm định t-test giữa VCB và BID: t-statistic = {t_stat:.2f}, p-value = {p_val:.2e}, df = {df:.0f}')


Kiểm định t-test giữa VCB và BID: t-statistic = 79.50, p-value = 0.00e+00, df = 2976


In [30]:
# Biểu đồ hộp (Boxplot) giá đóng cửa của từng cổ phiếu
fig = px.box(data, x='Ticker Symbol', y='Price', title='Biểu đồ hộp giá đóng cửa của từng cổ phiếu')
fig.show()

In [31]:
# Kiểm định ANOVA để so sánh giá đóng cửa của nhiều nhóm cổ phiếu
model = ols('Price ~ C(Q("Ticker Symbol"))', data=data).fit()
anova_results = anova_lm(model)
print('\nKết quả kiểm định ANOVA:')
print(anova_results)


Kết quả kiểm định ANOVA:
                           df        sum_sq       mean_sq            F  PR(>F)
C(Q("Ticker Symbol"))     4.0  3.122320e+12  7.805801e+11  7588.410917     0.0
Residual               7440.0  7.653139e+11  1.028648e+08          NaN     NaN


### Câu hỏi 3:

In [32]:
# Lọc dữ liệu cho cổ phiếu VCB
vcb_data = data[data['Ticker Symbol'] == 'VCB']
bid_data = data[data['Ticker Symbol'] == 'BID']
ctg_data = data[data['Ticker Symbol'] == 'CTG']
tcb_data = data[data['Ticker Symbol'] == 'TCB']
agr_data = data[data['Ticker Symbol'] == 'AGR']

# Sắp xếp dữ liệu theo ngày giảm dần và lấy dữ liệu của 10 tuần gần nhất (70 ngày gần nhất)
vcb_data = vcb_data.sort_values(by='Date', ascending=False).head(70)
bid_data = bid_data.sort_values(by='Date', ascending=False).head(70)
ctg_data = ctg_data.sort_values(by='Date', ascending=False).head(70)
tcb_data = tcb_data.sort_values(by='Date', ascending=False).head(70)
agr_data = agr_data.sort_values(by='Date', ascending=False).head(70)

unique_tickers = data['Ticker Symbol'].unique()

# Tạo biểu đồ nến
fig = go.Figure(data=[go.Candlestick(
    x=vcb_data['Date'],
    open=vcb_data['Open'],
    high=vcb_data['High'],
    low=vcb_data['Low'],
    close=vcb_data['Price']
)])

# Thêm biểu đồ nến của các cổ phiếu khác
fig.add_trace(go.Candlestick(
    x=bid_data['Date'],
    open=bid_data['Open'],
    high=bid_data['High'],
    low=bid_data['Low'],
    close=bid_data['Price'],
    visible=False
))

fig.add_trace(go.Candlestick(
    x=ctg_data['Date'],
    open=ctg_data['Open'],
    high=ctg_data['High'],
    low=ctg_data['Low'],
    close=ctg_data['Price'],
    visible=False
))

fig.add_trace(go.Candlestick(
    x=tcb_data['Date'],
    open=tcb_data['Open'],
    high=tcb_data['High'],
    low=tcb_data['Low'],
    close=tcb_data['Price'],
    visible=False
))

fig.add_trace(go.Candlestick(
    x=agr_data['Date'],
    open=agr_data['Open'],
    high=agr_data['High'],
    low=agr_data['Low'],
    close=agr_data['Price'],
    visible=False
))

# Cài đặt layout cho biểu đồ
fig.update_layout(
    title='Biểu đồ nến của cổ phiếu VCB',
    xaxis_title="Ngày",
    yaxis_title="Giá",
    xaxis_rangeslider_visible=False
)

fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                # {
                #     'method': 'update',
                #     'label': 'All',
                #     'args': [{'visible': [True] * len(unique_tickers)}, {
                #         'title': 'Biểu đồ nến của các cổ phiếu'
                #     }]
                # },
                # {
                #     'method': 'update',
                #     'label': 'None',
                #     'args': [{'visible': [False] * len(unique_tickers)}, {
                #         'title': 'Biểu đồ nến của các cổ phiếu'
                #     }]
                # },
                {
                    'method': 'update',
                    'label': 'VCB',
                    'args': [{'visible': list(unique_tickers == 'VCB')}, {
                        'title': 'Biểu đồ nến của cổ phiếu VCB'
                    }]
                },
                {
                    'method': 'update',
                    'label': 'BID',
                    'args': [{'visible': list(unique_tickers == 'BID')}, {
                        'title': 'Biểu đồ nến của cổ phiếu BID'
                    }]
                },
                {
                    'method': 'update',
                    'label': 'CTG',
                    'args': [{'visible': list(unique_tickers == 'CTG')}, {
                        'title': 'Biểu đồ nến của cổ phiếu CTG'
                    }]
                },
                {
                    
                    'method': 'update',
                    'label': 'TCB',
                    'args': [{'visible': list(unique_tickers == 'TCB')}, {
                        'title': 'Biểu đồ nến của cổ phiếu TCB'
                    }]
                },
                {
                    
                    'method': 'update',
                    'label': 'AGR',
                    'args': [{'visible': list(unique_tickers == 'AGR')}, {
                        'title': 'Biểu đồ nến của cổ phiếu AGR'
                    }]
                }
            ],
        }
    ]
)

# Hiển thị biểu đồ
fig.show()